In [1]:
!pip install huggingface_hub transformers protobuf sentencepiece accelerate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 385.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 280.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.8/792.8 kB 194.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 274.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
!pip install ../diffusers

Processing /workspace/diffusers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.32.0.dev0-py3-none-any.whl size=2899207 sha256=056d6a659c78e979328232f62f3daead22f5e2bc5535a981ac6d6c8e2b9411fd
  Stored in directory: /tmp/pip-ephem-wheel-cache-52i3m26f/wheels/84/89/b5/dd86d0e3409ce383621f20d1f706ebad7fe1c2ac2b338cea6a
Successfully built diffusers

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
## no_grad -> inference_mode found no speedup.
## Adding torch compile 57 sec is base. 39 sec now.
## caching the model output -> 29 sec

In [1]:
import torch
# from diffusers import FluxPipeline
from pipeline_flux import FluxPipeline
from huggingface_hub import hf_hub_download
base_model_id = "black-forest-labs/FLUX.1-dev"
repo_name = "ByteDance/Hyper-SD"
# Take 8-steps lora as an example
ckpt_name = "Hyper-FLUX.1-dev-8steps-lora.safetensors"
# Load model, please fill in your access tokens since FLUX.1-dev repo is a gated model.
pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [2]:
# pipe.load_lora_weights(hf_hub_download(repo_name, ckpt_name))
# pipe.fuse_lora(lora_scale=0.125)
pipe.to("cuda", dtype=torch.bfloat16)

FluxPipeline {
  "_class_name": "FluxPipeline",
  "_diffusers_version": "0.32.0.dev0",
  "_name_or_path": "black-forest-labs/FLUX.1-dev",
  "scheduler": [
    "diffusers",
    "FlowMatchEulerDiscreteScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "T5EncoderModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "T5TokenizerFast"
  ],
  "transformer": [
    "diffusers",
    "FluxTransformer2DModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [3]:
prompt = "A cat holding a sign that says hello world"

In [6]:
pipe.vae = torch.compile(pipe.vae)

In [7]:
%%time
image = pipe(
    prompt,
    height=1024,
    width=1024,
    guidance_scale=3.5,
    num_inference_steps=28,
    max_sequence_length=512,
    generator=torch.Generator("cuda").manual_seed(0),
    compile=False,
    fullgraph=False,
    cache=False,
    joint_attention_kwargs={}
).images[0]

  0%|          | 0/28 [00:00<?, ?it/s]

CPU times: user 14.6 s, sys: 15.9 s, total: 30.5 s
Wall time: 30.2 s


| Measured on | Resolution, Steps | Time (sec)<sup>#</sup> |
|----------|----------|----------|
| Default Pipeline | 1024px, 28 | 30.5 |
| With `torch.compile` | 1024px, 28 | 22<sup>*</sup>|
| With `torch.compile` and model pred caching | 1024px, 28 | 17.4<sup>*</sup> |
| HyperSD LoRA With `torch.compile` | 1024px, 8 | 6.51<sup>*</sup> |
| HyperSD LoRA With `torch.compile` and model pred caching | 1024px, 8 | 5.73<sup>*</sup> |
| Default Pipeline | 512px, 28 | 9.91 |
| With `torch.compile` | 512px, 28 | 7.09<sup>*</sup> |
| With `torch.compile` and model pred caching | 512px, 28 | 5.61<sup>*</sup> |
| HyperSD LoRA With `torch.compile` | 512px, 8 | 2.16<sup>*</sup> |
| HyperSD LoRA With `torch.compile` and model pred caching | 512px, 8 | 1.93<sup>*</sup> |

<sup>*</sup> Compiliation time not included. Only transformer was compiled. Compiling other parts will make it faster.<br>
<sup>#</sup> Wall time as measured with %%time magic. Not the tqdm measurement. The times are meant to make relative sense rather than absolute measures.<br>

Environment `Nvidia A40 torch 2.4.1 CuDA 12.4`

In [9]:
!pip list | grep torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


torch                             2.4.1+cu124
torchaudio                        2.4.1+cu124
torchvision                       0.19.1+cu124
